## 0. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
from collections import Counter

from imblearn.over_sampling import SMOTE
import matplotlib.ticker as ticker

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix, f1_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1. Loading Data

In [ ]:
# from google.colab import files
# data_to_load = files.upload()

In [ ]:
import io
#df = pd.read_csv(io.BytesIO(data_to_load['RTA Dataset.csv']))
df=pd.read_csv(r"/content/RTA Dataset.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sample(5)

## 2. Exploratory Data Analysis

### 2.1. EDA - Basic

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.isnull().all().sum() # No features are with all missing values

In [ ]:
df.isnull().any().sum() # 16 Features with some missing values

In [ ]:
missing_value_df = pd.DataFrame({'Missing Counts' : df.isnull().sum(), 'percent_missing': round(df.isnull().sum() * 100 / len(df), 2)})
missing_value_df.sort_values('percent_missing', ascending = False , inplace=True)

missing_value_df #What are those features with some missing values and their proportion to the total.

In [ ]:
df.duplicated().sum() #No duplicates

In [ ]:
df.describe().T

In [ ]:
df.describe(include="object").T

In [ ]:
# Insights : 

# Most accidents are of severity : "Slight Injury" and happened due vehicles collision.
# Root Cause Of the Collision : No Distancing between vehicles.
# Driver Stats : Male, Age Band [18-30], Junior High School, Employee, 5-10 Yrs Exp.
# Vehicle Stats : Automobile, Private Vehicle, No Defect, Service Year - Unknown.
# Time Stats : Friday Evening [3:30 PM], Daylight , Normal Weather
# Place Stats : Other, Two-way lanes (divided with broken lines road marking), Y Shape Junction
# Road Stats : Tangent road with flat terrain, Asphalt roads, Dry

### 2.2. EDA - Advanced

#### 2.2.1. EDA - Advanced - Numerical features analysis

In [ ]:
df.hist(layout=(1,2), figsize=(6,3))
plt.show()

#Insight : Majority of accidents are by 1 or 2 vehicles and casualties are <=3.

In [ ]:
sns.boxplot(data=df, y='Number_of_vehicles_involved')
plt.show()

# Insights : Majorly, 2 vehicles. Outliers are present.

In [ ]:
sns.boxplot(data=df, y='Number_of_casualties')
plt.show()

# Insights : 50% of data with, 2 casualties. Outliers are present.

In [ ]:
sns.pairplot(data = df)
plt.show() #No Correlation between Number_of_vehicles_involved & Number_of_Casualties.

In [ ]:
sns.pairplot(data = df, vars = ['Number_of_vehicles_involved', 'Number_of_casualties'], hue = 'Accident_severity', markers=["o", "s", "D"], height = 3, kind = 'reg')
plt.show() 

#No Correlation between Number_of_vehicles_involved & Number_of_Casualties. [May be there is a pattern in Fatal Injury, Less No.of.vehicles involved, fatal inuries]

In [ ]:
sns.heatmap(df[['Number_of_vehicles_involved','Number_of_casualties']].corr(), annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('dark')
sns.histplot(data = df, x= 'Number_of_vehicles_involved', hue = 'Accident_severity', binwidth=1, multiple="stack") #or use 'bins'
plt.show()

# Insights : More Fatal Injuries when No. of Vehicles Involved = 2 [May be due vehicle-vehicle collision]

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('dark')
sns.histplot(data = df, x= 'Number_of_casualties', hue = 'Accident_severity', binwidth=1, multiple="stack") #or use 'bins'
plt.show()

# Insights : Majorly ONE Person has injured, with Fatal Injury. [May be the Rider]

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('dark')
sns.histplot(data = df, x= 'Number_of_vehicles_involved', hue = 'Number_of_casualties', binwidth=1, multiple="stack") #or use 'bins'
plt.show()

# Insights : More Casualities, when no of vehicles involved <= 3, Majorly when 2. [Vehicles Collision?]

#### 2.2.2 EDA - Advanced - Categorical Features Analysis

In [ ]:
plt.figure(figsize=(10,7))
plt.pie(x=df['Accident_severity'].value_counts().values,
        labels=df['Accident_severity'].value_counts().index,
        autopct='%2.2f%%')
plt.show() # 80% of Accidents ended up with Slight Injury.

In [ ]:
df['Time']= pd.to_datetime(df['Time'])

df['Hour']= df['Time'].dt.hour
df['Minute']= df['Time'].dt.minute


In [ ]:
df.drop(['Time'], axis=1, inplace = True)

In [ ]:
# for column in df.select_dtypes(include=['object']).columns[:-1] :
#   sns.countplot(data=df, y= column) #linewidth=2, edgecolor=sns.color_palette("dark", 3)) #palette="rainbow"
#   plt.xticks(rotation=45, horizontalalignment='right', fontweight='light')
#   plt.show() Note : DF.describe explained the same

for column in ['Hour', 'Minute']:
  plt.figure(figsize=(10,20))
  sns.countplot(data=df, y= column) #linewidth=2, edgecolor=sns.color_palette("dark", 3)) #palette="rainbow"
  plt.xticks(rotation=90, horizontalalignment='right', fontweight='light')
  plt.show()


#Insights : 

#1. Most Accidents : Mornings - 7 to 10 ; Evenings - 3 to 7 
#2. Every 5th Minute, Rise in accidents counts [Coincidental or convenience]

In [ ]:
for column in df.select_dtypes(include=['object']).columns[:-1] :
  sns.countplot(data=df, y= column, hue = 'Accident_severity') #linewidth=2, edgecolor=sns.color_palette("dark", 3)) #palette="rainbow"
  plt.xticks(rotation=45, horizontalalignment='right', fontweight='light')
  plt.show()

for column in ['Hour', 'Minute']:
  plt.figure(figsize=(10,20))
  sns.countplot(data=df, y= column, hue = 'Accident_severity') #linewidth=2, edgecolor=sns.color_palette("dark", 3)) #palette="rainbow"
  plt.xticks(rotation=90, horizontalalignment='right', fontweight='light')
  plt.show()


#Insights : 

#1. More Fatal Injury accidents are on weekends [Sat & Sun] [May be due Drink & Drive?]
#2. More Serious Injury & Fatal Injury accidents are with drivers fall under Age Band [18-30 & 31-50] 
#3. Very Less Serious Injury & No Fatal Injury, when Female Driver but that's not when Male driver [May be due Speed?]
#4. Serious & Fatal Injuries are caused by Drivers with Education Levels : Elementary School, Junior High School, High School
#5. More Serious & Fatal Injuries are caused by Drivers - who are employees.
#6. Across all Driving Experience bands, Accidents. Majorly, Fatal Injuries are by Drivers with 2-10 Yrs Exp.
#7. Fatal Injuries are caused by majorly Automobiles. More Serious Injuries are caused by Automobiles and Lorry/Heavy Vehicles.
#8. More Serious & Fatal Injuries are caused by Private Owned Vehicles.
#9. Majority of Accidents had happened in Areas : Other + Residential + Office and even Fatal Injuries.
#10. Fatal Injuries happened in Undivided Two way or Two Way broken lanes.
#11. Fatal Injuries happened in Tangent Roads with Flat Terrains.
#12. Serious & Fatal Injuries happened in No Juntion or Y junction or Crossing.
#13. Serious & Fatal Injuries happened while Moving backward or chnaging lanes.
#14. Serious & Fatal Injuries happened during 3 PM To 10 PM and then 3-4AM in the morning.
#15. Serious & Fatal Injuries happened when pedestrain movement in "Not a Pedestrain" zone.

In [ ]:
for column in df.select_dtypes(include=['object']).columns[:-1] :
  # creating a facet grid with Target
  grid = sns.FacetGrid(data=df, col='Accident_severity', height=4, aspect=1, sharey=False)
  # mapping bar plot and the data on to the grid
  grid.map(sns.countplot, column) #, palette=['blue', 'green', 'orange']
  grid.set_xticklabels(rotation=90) #plt.xticks(rotation=45, horizontalalignment='right', fontweight='light')
  plt.show()

In [ ]:
data = df.copy()

## 3. Data Pre-Prcoessing

In [ ]:
df = data.copy()

### 3.1. Feature Value Transformations [Split/Extraction]

In [ ]:
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Long lorry', 'Lorry (41?100Q)', 'Lorry (11?40Q)', 'Pick up upto 10Q'], 'Lorry')

df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Bajaj', 'Motorcycle', 'Bicycle'], 'Bike')

df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Public (12 seats)', 'Public (13?45 seats)', 'Public (> 45 seats)'], 'Public')

df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Automobile', 'Taxi', 'Turbo', 'Special vehicle', 'Stationwagen'], 'Car')

df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Other', np.NaN], 'Other') 

df['Owner_of_vehicle'] = df['Owner_of_vehicle'].replace(['Other', np.NaN], 'Other') 

df['Area_accident_occured'] = df['Area_accident_occured'].replace(['Other', 'Unknown', np.NaN], 'Other') 
df['Area_accident_occured'] = df['Area_accident_occured'].replace(['Rural village areasOffice areas'], 'Rural village areas') 

df['Lanes_or_Medians'] = df['Lanes_or_Medians'].replace(['Other', 'unknown', np.NaN], 'Other') 

df['Types_of_Junction'] = df['Types_of_Junction'].replace(['Other', 'Unknown', np.NaN], 'Other') 

df['Road_surface_type'] = df['Road_surface_type'].replace(['Other', np.NaN], 'Other') 

df['Road_surface_conditions'] = df['Road_surface_conditions'].replace(['Flood over 3cm. deep', 'Wet or damp', 'Snow'], 'Wet') 

df['Light_conditions'] = df['Light_conditions'].replace(['Daylight', 'Darkness - lights lit'], 'Light') 
df['Light_conditions'] = df['Light_conditions'].replace(['Darkness - no lighting', 'Darkness - lights unlit'], 'Dark') 

df['Weather_conditions'] = df['Weather_conditions'].replace(['Cloudy', 'Raining', 'Raining and Windy','Cloudy', 'Windy', 'Snow', 'Fog or mist'], 'Abnormal') 
df['Weather_conditions'] = df['Weather_conditions'].replace(['Other', 'Unknown'], 'Normal') 

df['Type_of_collision'] = df['Type_of_collision'].replace(['Other', 'Unknown', np.NaN], 'Other') 

df['Vehicle_movement'] = df['Vehicle_movement'].replace(['Other', np.NaN], 'Other') 

df['Work_of_casuality'] = df['Work_of_casuality'].replace(['Other', 'Unknown', np.NaN], 'Other') 

df['Fitness_of_casuality'] = df['Fitness_of_casuality'].replace(['Other', np.NaN], 'Other') 
df['Fitness_of_casuality'] = df['Fitness_of_casuality'].replace(['NormalNormal'], 'Normal') 

df['Cause_of_accident'] = df['Cause_of_accident'].replace(['Other', 'Unknown'], 'Other') 

df['Pedestrian_movement'] = df['Pedestrian_movement'].replace(["Crossing from driver's nearside", 'Crossing from nearside - masked by parked or statioNot a Pedestrianry vehicle', 'Crossing from offside - masked by  parked or statioNot a Pedestrianry vehicle'], 'Crossing') 
df['Pedestrian_movement'] = df['Pedestrian_movement'].replace(["In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing)", 'In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing) - masked by parked or statioNot a Pedestrianry vehicle', 'Walking along in carriageway, back to traffic', 'Walking along in carriageway, facing traffic'], 'In carriageway') 

df['Cause_of_accident'] = df['Cause_of_accident'].replace(['Changing lane to the right', 'Changing lane to the left', 'Driving to the left'], 'Changing lanes') 
df['Cause_of_accident'] = df['Cause_of_accident'].replace(['Driving under the influence of drugs'], 'Drunk driving') 
df['Cause_of_accident'] = df['Cause_of_accident'].replace(['Driving at high speed'], 'Overspeed') 


In [ ]:
min = list(range(5,56, 5))
def convert_minutes(x: int):
    for m in min:
        if x % m == x and x > m-5:
            return m
        if x in [56,57,58,59]:
            return 0
        if x in min+[0]:
            return x

df['Minute'] = df['Minute'].apply(lambda x: convert_minutes(x))

###3.2. Missing Values Treatment

In [ ]:
columns_to_drop = ['Defect_of_vehicle', 'Service_year_of_vehicle','Work_of_casuality','Fitness_of_casuality']

df.drop(columns_to_drop, axis = 1, inplace = True) #20% + missing values

In [ ]:
df.replace('Unknown', np.NaN, inplace = True)
df.replace('unknown', np.NaN, inplace = True)
# df.replace('na', np.NaN, inplace = True)

In [ ]:
columns_with_missing_values = missing_value_df[missing_value_df.percent_missing !=0.00].index[4:]

for column in columns_with_missing_values:
  #df[column].fillna(df[column].mode()[0], inplace=True) #Mode Imputation, as all categorical features.
  df[column].fillna('Unknown', inplace=True) # Constant Imputation, as all categorical features.

### 3.3. Encoding Techniques

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()

# df = df.apply(encoder.fit_transform)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['Accident_severity'] = encoder.fit_transform(df['Accident_severity'])

categorical_features = df.select_dtypes(include='object').columns

for feature in categorical_features : 
  df = pd.get_dummies(df, prefix = ['Temp'+'_'+feature], columns = [feature], drop_first = False)

In [ ]:
# plt.figure(figsize = [25,15])
# sns.heatmap(df.corr(), annot = True, mask = np.triu(np.ones_like(df.corr(), dtype=bool)))

### 3.4. Imbalanced Data Treatment

In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#sampler = RandomOverSampler()

# from imblearn.over_sampling import SMOTE
# sampler = SMOTE()


# X = df.drop('Accident_severity', axis=1)
# y = df['Accident_severity']

# X,y = sampler.fit_resample(X,y)

### 3.5. Skewed Data Treatment

### 3.6. Outlier Treatment

### 3.7. Feature Scaling

### 3.8. Binning

### 3.9. Grouping Operations

### 3.10. Dimensionality Reduction

### 3.11. Multi-Colinearity Treatment

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = df.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
  
print(vif_data) #VIF =1 -> No Correlation VIF = 1 to 5 -> Medium Correlation VIF > 5 -> Severe Correlation

#Refer Article : https://www.statology.org/multicollinearity-regression/ to treat.

In [ ]:
print(vif_data[vif_data['VIF'] >= 5])

## 4. ML Modelling

### 4.1. Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Accident_severity', axis=1)
y = df['Accident_severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1, stratify = y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from imblearn.over_sampling import SMOTE
sampler = SMOTE()

X_train,y_train = sampler.fit_resample(X_train,y_train)

In [ ]:
from collections import Counter

counter = Counter(y_train)

print("=============================")

for k,v in counter.items():
    per = 100*v/len(y_train)
    print(f"Class= {k}, n={v} ({per:.2f}%)")

### 4.2. Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from matplotlib import pyplot

def get_features_scores(X_train, y_train):
	fs = SelectKBest(score_func=chi2, k='all')
	fs.fit(X_train, y_train)
	return fs

fs = get_features_scores(X_train, y_train)

# for i in range(len(fs.scores_)):
# 	print('Feature %d: %f' % (i, fs.scores_[i]))
# # plot the scores
# pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
# pyplot.show()

In [ ]:
scores = pd.Series(fs.scores_)

In [ ]:
def select_features(X_train, y_train, X_test, k):
	fs = SelectKBest(score_func=chi2, k=k)
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs

X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, 10) #scores[scores > scores.mean()].shape[0]

### 4.3. Training the Model

In [ ]:
model = LogisticRegression(random_state=1)
model.fit(X_train_fs, y_train)

### 4.4. Evaluating the Model

In [ ]:
y_pred = model.predict(X_test_fs)

print(f1_score(y_test, y_pred, average='weighted'))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

### 4.5. AutoML

In [ ]:
# !pip install TPOT

In [ ]:
# from sklearn.model_selection import RepeatedStratifiedKFold
# from tpot import TPOTClassifier

In [ ]:
# # define model evaluation
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

# #Custom Scoring function
# #f1 = make_scorer(f1_score, average='weighted')

# # define search
# model = TPOTClassifier(generations=10, population_size=100, cv=cv, scoring= 'f1_weighted', verbosity=2, random_state=1, n_jobs=-1, early_stop = 50, periodic_checkpoint_folder="tpot_checkpoint.txt", memory = 'auto') #, config_dict='TPOT light'
# # perform the search
# model.fit(X_train_fs, y_train)
# # export the best model
# model.export('tpot_best_model.py')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# # copy it there
# !cp /content/tpot_best_model.py /content/drive/MyDrive

### 4.6. Re-Fit with best Model from TPOT

In [ ]:
# exported_pipeline = GradientBoostingClassifier(learning_rate=0.1, max_depth=10, max_features=0.4, min_samples_leaf=2, min_samples_split=5, n_estimators=100, subsample=0.45)
# # Fix random state in exported estimator
# if hasattr(exported_pipeline, 'random_state'):
#     setattr(exported_pipeline, 'random_state', 1)

# exported_pipeline.fit(X_train_fs, y_train)

In [ ]:
# y_pred = exported_pipeline.predict(X_test_fs)

# print(f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# confusion_matrix(y_test, y_pred)

In [ ]:
# print(classification_report(y_test, y_pred))

### 4.7. Hyperparameter Tuning

In [ ]:
# from sklearn.model_selection import GridSearchCV

# p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000]}

# tuning = GridSearchCV(estimator = GradientBoostingClassifier(max_depth=10, max_features=0.4, min_samples_leaf=2, min_samples_split=5, subsample=0.45), 
#             param_grid = p_test3, scoring='f1_weighted', n_jobs=-1, cv=5)

# tuning.fit(X_train_fs,y_train)

# tuning.grid_scores_, tuning.best_params_, tuning.best_score_

### 4.8. Save Model

In [ ]:
!pip install pipreqs -q

In [ ]:
!pipreqs .

In [ ]:
# !pip install streamlit -q

In [ ]:
# !python --version

In [ ]:
import joblib


joblib.dump(model, "model.joblib")